In [1]:
import json
import logging
import re
import warnings
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Generator, Literal, Type, TypeVar

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)


def create_path(path: str | Path) -> None:
    """
    Create parent directories for the given path if they don't exist.

    Parameters
    ----------
    path : str | Path
        The file path for which to create parent directories.

    """
    Path(path).parent.mkdir(parents=True, exist_ok=True)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/MyProjects/batch-process


In [4]:
from schemas import EmailSchema
from schemas.db_models import EmailLog, get_db_session, init_db

Connected to 'test' environment database.


In [5]:
init_db()

## [Docs](https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html)

### Insert

In [6]:
input_data: EmailSchema = EmailSchema(
    recipient="ayo@example.com",
    subject="test subject",
    body="test body",
    status="pending",
)
input_data_2: EmailSchema = EmailSchema(
    recipient="emeka2@example.com",
    subject="test!!!",
    body="this is an example body",
    status="pending",
)
console.print((input_data, input_data_2))

(
    EmailSchema(
        recipient='ayo@example.com',
        subject='test subject',
        body='test body',
        status='pending',
        created_at=datetime.datetime(2025, 7, 13, 0, 51, 50, 725709),
        sent_at=None
    ),
    EmailSchema(
        recipient='emeka2@example.com',
        subject='test!!!',
        body='this is an example body',
        status='pending',
        created_at=datetime.datetime(2025, 7, 13, 0, 51, 50, 725780),
        sent_at=None
    )
)

In [7]:
with get_db_session() as session:
    data_dict = input_data.to_data_model_dict()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

{
    'id': 1,
    'recipient': 'ayo@example.com',
    'subject': 'test subject',
    'status': 'pending',
    'sent_at': None,
    'created_at': '2025-07-13T00:51:50.725709'
}

In [8]:
with get_db_session() as session:
    data_dict = input_data_2.to_data_model_dict()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

{
    'id': 2,
    'recipient': 'emeka2@example.com',
    'subject': 'test!!!',
    'status': 'pending',
    'sent_at': None,
    'created_at': '2025-07-13T00:51:50.725780'
}

### Select

In [9]:
from sqlalchemy import delete, select, update

with get_db_session() as session:
    statement = select(EmailLog).where(EmailLog.id == 1, EmailLog.status == "pending")
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

{
    'id': 1,
    'recipient': 'ayo@example.com',
    'subject': 'test subject',
    'status': 'pending',
    'sent_at': None,
    'created_at': '2025-07-13T00:51:50.725709'
}

### [Update](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [11]:
with get_db_session() as session:
    statement = update(EmailLog).where(EmailLog.id == 1).values(status="sent")
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).all()
    output_data = {key: getattr(record, key) for key in record.output_fields()}

console.print(output_data)

AttributeError: 'list' object has no attribute 'output_fields'

### Delete

In [ ]:
with get_db_session() as session:
    statement = delete(EmailLog).where(EmailLog.id == 1).values(status="sent")
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog).where(EmailLog.id == 1)
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}

console.print(output_data)